# Decision Tree Example

In [1]:
import numpy as np
import pandas as pd 

# 모델 라이브러리 선언
from sklearn import datasets, tree

# 모델 정확도 라이브러리 선언
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# 시각화 라이브러리 선언
import matplotlib.pyplot as plt

### 1. 분석데이터 로딩

In [2]:
#CSV 파일을 읽어 DataFrame 변수에 저장하기
featuresData = pd.read_csv("../dataset./feature_regression_example.csv")

In [3]:
featuresData.corr()

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT
YEARWEEK,1.000000,0.961051,0.213022,0.037392,-0.030681,0.300320
YEAR,0.961051,1.000000,-0.065302,-0.048803,0.067443,0.208435
WEEK,0.213022,-0.065302,1.000000,0.307541,-0.349205,0.347462
QTY,0.037392,-0.048803,0.307541,1.000000,-0.544920,0.712772
HCLUS,-0.030681,0.067443,-0.349205,-0.544920,1.000000,-0.552991
PRO_PERCENT,0.300320,0.208435,0.347462,0.712772,-0.552991,1.000000


In [4]:
featuresData.head(1)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442


### 2.데이터 형 변환

In [5]:
featuresData.dtypes

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK              int64
QTY               int64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
dtype: object

In [6]:
#형변환
featuresData[['WEEK','QTY','PRO_PERCENT']]=featuresData[['WEEK','QTY','PRO_PERCENT']].astype('float64')
featuresData.dtypes

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK            float64
QTY             float64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
dtype: object

### 3. 문자데이터 코드변환(Vector연산)

In [7]:
#case when 
############################
def codeConversion(df):
    if df == "Y":
        return 1
    else:
        return 0

In [8]:
featuresData['PROMOTIONCODE'] = featuresData['PROMOTION'].apply(codeConversion)
featuresData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2.0,968.0,N,4,Y,0.209442,1
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3.0,1209.0,N,4,Y,0.208155,1
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4.0,1810.0,Y,2,Y,0.208155,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5.0,1773.0,N,4,Y,0.208155,1


In [9]:
featuresData['HOLIDAYCODE'] = featuresData['HOLIDAY'].apply(codeConversion)
featuresData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2.0,968.0,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3.0,1209.0,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4.0,1810.0,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5.0,1773.0,N,4,Y,0.208155,1,0


In [10]:
featuresData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2.0,968.0,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3.0,1209.0,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4.0,1810.0,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5.0,1773.0,N,4,Y,0.208155,1,0


### 4. 데이터 셋 분리

In [11]:
predictStd = 201630

In [12]:
#where between and (조건 절)
trainingData = featuresData.query('YEARWEEK <= @predictStd')
trainingData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1.0,1225.0,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2.0,968.0,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3.0,1209.0,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4.0,1810.0,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5.0,1773.0,N,4,Y,0.208155,1,0


In [13]:
#where between and (조건 절)
testData = featuresData.query('YEARWEEK > @predictStd')
testData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
83,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201631,2016,31.0,1522.0,N,4,Y,0.280258,1,0
84,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201632,2016,32.0,2100.0,N,4,Y,0.280258,1,0
85,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201633,2016,33.0,43.0,N,4,N,0.000000,0,0
86,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201634,2016,34.0,1700.0,Y,1,Y,0.308584,1,1
87,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201635,2016,35.0,1514.0,Y,1,Y,0.308584,1,1


In [14]:
trainingData_feature = trainingData[['WEEK','PRO_PERCENT','HOLIDAYCODE']]

In [15]:
trainingData_label = trainingData[['QTY']]

In [16]:
testData_feature = testData[['WEEK','PRO_PERCENT','HOLIDAYCODE']]

In [17]:
testData_label = testData[['QTY']]

In [18]:
# from sklearn import tree
#clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

### 5. 모델선언 및 예측

In [22]:
### Extract Coefficient 머신러닝!!

In [23]:
model = model_method.fit(trainingData_feature, trainingData_label)

In [24]:
predict = model.predict(testData_feature)

In [25]:
predict

array([2240., 2240.,  147., 1318., 1318., 1318., 2240., 2240., 2240.,
       2240., 2671., 2240.,  230., 1454., 1454., 2033., 2033., 3691.,
       1758., 1758., 1758., 1758.,  350.,  301.])

In [26]:
testData_label.head()

,QTY
83,1522.0
84,2100.0
85,43.0
86,1700.0
87,1514.0


### 6. 데이터 정리

In [27]:
type(predict)

numpy.ndarray

In [28]:
predictData = pd.DataFrame(predict)

In [29]:
predictData.head()

,0
0,2240.0
1,2240.0
2,147.0
3,1318.0
4,1318.0


In [30]:
predictData.columns = ["PREDICT"]

In [31]:
testData.head(1)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
83,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201631,2016,31.0,1522.0,N,4,Y,0.280258,1,0


In [32]:
testData.reset_index(drop=True, inplace=True)

In [33]:
testData.head(2)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201631,2016,31.0,1522.0,N,4,Y,0.280258,1,0
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201632,2016,32.0,2100.0,N,4,Y,0.280258,1,0


In [34]:
predictData.reset_index(drop=True, inplace=True)

In [35]:
#testData_feature.reset_index(drop=True, inplace=True)

In [36]:
#cncatenate labels to df as a new column / column binding
finalDf = pd.concat([testData, predictData], axis = 1)

In [37]:
finalDf.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTIONCODE,HOLIDAYCODE,PREDICT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201631,2016,31.0,1522.0,N,4,Y,0.280258,1,0,2240.0
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201632,2016,32.0,2100.0,N,4,Y,0.280258,1,0,2240.0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201633,2016,33.0,43.0,N,4,N,0.000000,0,0,147.0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201634,2016,34.0,1700.0,Y,1,Y,0.308584,1,1,1318.0
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201635,2016,35.0,1514.0,Y,1,Y,0.308584,1,1,1318.0


In [38]:
finalDf.to_csv("./dt_result_20181022.csv")

In [39]:
pwd

'C:\\Users\\hk2\\Python_CJ_PROF\\Session01 - Why Python for Data Analysis'

### 7. 정확도 측정

In [40]:
mean_absolute_error(finalDf['QTY'], finalDf['PREDICT'])  

438.625

In [41]:
mean_squared_error(finalDf['QTY'], finalDf['PREDICT'])  

360522.2916666667

In [42]:
r2_score(finalDf['QTY'], finalDf['PREDICT'])  

0.4124315318636007

In [43]:
finalDf.to_csv("./decisiontree_result.csv")